In [1]:
# Python Descriptors - implementing methods of the Descriptor Protocol
########################################################################

class print_attr():
    def __get__(self, obj, type=None) -> object:
        print("retrieving value by accessing the attribute")
        return 45

    def __set__(self, obj, value) -> None:
        print("setting the value by accessing the attribute")
        raise AttributeError("Value could not be changed")

class Demo():
    myAttr = print_attr()


In [2]:
demo_obj = Demo()
var = demo_obj.myAttr
print(var)


retrieving value by accessing the attribute
45


In [13]:
# 1.1.1 Non-data descriptors
##############################

class NonDataDescriptor:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return 45

class ClientClass:
    desc = NonDataDescriptor()


In [14]:
myObj = ClientClass()
myObj.desc

45

In [15]:
myObj.desc = 25
myObj.desc

25

In [16]:
vars(myObj)

{'desc': 25}

In [3]:
# 1.1.2 Data Descriptors - descriptor that now implements the __set__ method 
###############################################################################

class DataDescriptor:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return 45

    def __set__(self, instance, value):
        print("%s.descriptor set to %s", instance, value)
        instance.__dict__["desc"] = value

class ClientClass:
    desc = DataDescriptor()


In [4]:
myObj = ClientClass()
myObj.desc

45

In [20]:
myObj.desc = 22
myObj.desc

%s.descriptor set to %s <__main__.ClientClass object at 0x0000025E431F8EE0> 22


45

In [21]:
vars(myObj)

{'desc': 22}

In [22]:
myObj.__dict__["desc"]

22

In [23]:
del myObj.desc

AttributeError: __delete__

In [5]:
# The look up chain and attribute access
###########################################

class Animal():
    can_fly = False
    num_legs = 0

class Giraffe(Animal):
    num_legs = 4

    def __init__(self, name):
        self.name = name

my_pet = Giraffe("Jerry")
print(my_pet.__dict__)
print(type(my_pet).__dict__)


{'name': 'Jerry'}
{'__module__': '__main__', 'num_legs': 4, '__init__': <function Giraffe.__init__ at 0x0000013CB1AD9280>, '__doc__': None}


In [6]:
# what happens internally when an attribute is accessed
#########################################################

class Animal(object):
    can_fly = False
    num_legs = 0

class Dog(Animal):
    num_legs = 4

    def __init__(self, name):
        self.name = name

my_pet = Dog("Enzo")


In [7]:
print(my_pet.name)
print(my_pet.num_legs)
print(my_pet.can_fly)


Enzo
4
False


In [8]:
# Re-written Pythonic version of above example 
#################################################

class Animal():
    can_fly = False
    num_legs = 0

class Dog(Animal):
    num_legs = 4

    def __init__(self, name):
        self.name = name

my_pet = Dog("Enzo")

print(my_pet.__dict__['name'])
print(type(my_pet).__dict__['num_legs'])
print(type(my_pet).__base__.__dict__['can_fly'])


Enzo
4
False


In [9]:
# 1.1.3 Property Objects are Data Descriptors
###############################################

class Demo():
    @property
    def attr_one(self) -> object:
        print("Getting the value from property")
        return 45

    @attr_one.setter
    def attr_one(self, value) -> None:
        print("Setting the value for the attribute")
        raise AttributeError("RO: Value change not allowed.")

myObj = Demo()
result = myObj.attr_one
print(result)


Getting the value from property
45


In [10]:
# The above can be written without using the decorators 
# explicitly and simply using functions instead
########################################################

class Demo():
    def get(self) -> object:
        print("Getting the value from the attribute")
        return 45

    def set(self, value) -> None:
        print("Setting the value for the attribute")
        raise AttributeError("RO: Value change not allowed.")

    attr_one = property(get, set)

myObj = Demo()
result = myObj.attr_one
print(result)

Getting the value from the attribute
45


In [11]:
# 1.1.4	Creating a Cached Property with Descriptors
#####################################################

class CachedProperty: 
    def __init__(self, func): 
        self.func = func 
 
    def __get__(self, instance, owner): 
        func_result = self.func(instance)
        instance.__dict__[self.func.__name__] = func_result
        return func_result

class Demo:
    def get_value(self):
        print('Using the method to compute the value!')
        return 84

    value = CachedProperty(get_value)
    # NOTE: You can use it as a decorator too!!


In [12]:
demo = Demo()
vars(demo)

{}

In [13]:
demo.value

Using the method to compute the value!


84

In [14]:
vars(demo)

{'get_value': 84}

In [15]:
demo.value

Using the method to compute the value!


84

In [17]:
# 1.1.5 Python Descriptors in Methods and Functions
#####################################################

import types

class Function(object):
    def __get__(self, obj, objtype=None):
        "Simulate func_descr_get() in Objects/funcobject.c"
        if obj is None:
            return self
        return types.MethodType(self, obj)


In [18]:
class StaticMethod(object):
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"
    def __init__(self, f):
        self.f = f

    def __get__(self, obj, objtype=None):
        return self.f


In [19]:
class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"
    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)
        def newfunc(*args):
            return self.f(klass, *args)
        return newfunc


In [20]:
# 1.1.6	Writable Data Attributes
#################################

class System():
    def __init__(self):
        self._os = None

    def get_os(self):
        return self._os

    def set_os(self, os):
        if 'RHEL' not in os:
            raise AttributeError('We only support RHEL 7 and above')
        self._os = os
        print('{} installation complete!'.format(os))


    os = property(get_os, set_os)


In [ ]:
server = System()
server.os = 'MacOS X'

In [24]:
class Property():
    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)


In [50]:
# The methods in the Property class act as decorators here - a better version of the above

class System():
    def __init__(self):
        self._os = None

    @property
    def os(self):
        return self._os

    @os.setter
    def os(self, os):
        if 'RHEL' not in os:
            raise AttributeError('We only support RHEL 7 and above')
        self._os = os
        print('{} installation complete!'.format(os))


In [25]:
# 1.1.9 Slots - Descriptors are used topersist/retrieve values from without a __dict__
#######################################################################################

class Car:
    __slots__ = ("make", "model")
    def __init__(self, make, model):
        self.make = make
        self.model = model

    def __repr__(self):
        return f"{self.__class__.__name__}({self.make}, {self.model})"


In [26]:
# 1.1.11 Descriptors are instantiated only once per Class. 
###########################################################

class SingleDigitNumber:
    def __init__(self):
        self.number = 0

    def __get__(self, myObj, type=None) -> object:
        return self.number

    def __set__(self, myObj, number) -> None:
        if int(number) != number or number > 9 or number < 0:
            raise AttributeError("Enter a valid 1-digit number")
        self.number = number

class Demo:
    number = SingleDigitNumber()


In [28]:
object_one = Demo()
object_two = Demo()

object_one.number = 8
print(object_one.number)
print(object_two.number)

object_three = Demo()
print(object_three.number)


8
8
8


In [29]:
# Alternative - persist the descriptor values for all the objects in a dictionary
###################################################################################

class SingleDigitNumber:
    def __init__(self):
        self.number = {}

    def __get__(self, myObj, type=None) -> object:
        try:
            return self.number[myObj]
        except:
            return 0

    def __set__(self, myObj, number) -> None:
        if int(number) != number or number > 9 or number < 0:
            raise AttributeError("Enter a valid 1-digit number")
        self.number[myObj] = number

class Demo:
    number = SingleDigitNumber()

object_one = Demo()
object_two = Demo()

object_one.number = 8
print(object_one.number)
print(object_two.number)

object_three = Demo()
print(object_three.number)


8
0
0


In [32]:
# Implementation where we automatically assign the name parameter, 
# when you instatiate the decorator
###################################################################

class SingleDigitNumber:
    def __init__(self, number):
        self.number = number

    def __get__(self, myObj, type=None) -> object:
        return myObj.__dict__.get(self.number) or 0

    def __set__(self, myObj, number) -> None:
        myObj.__dict__[self.number] = number

class Demo:
    number = SingleDigitNumber("number")

object_one = Demo()
object_two = Demo()

object_one.number = 8
print(object_one.number)
print(object_two.number)

object_three = Demo()
print(object_three.number)

8
0
0


In [33]:
# 1.1.13 Accessing Descriptor Methods
#######################################

from weakref import WeakKeyDictionary

class CallbackProperty(object):
    """A property that will alert observers when upon updates"""
    def __init__(self, default=None):
        self.default = default
        self.dataset = WeakKeyDictionary()
        self.callbacks = WeakKeyDictionary()
        
    def __get__(self, myObject, owner):
        if myObject is None:
            return self        
        return self.dataset.get(myObject, self.default)
    
    def __set__(self, myObject, value):
        for callback in self.callbacks.get(myObject, []):
            # alert callback function of new value
            callback(value)
        self.dataset[myObject] = value
        
    def register_callback(self, myObject, callback):
        if myObject not in self.callbacks:
            self.callbacks[myObject] = []
        self.callbacks[myObject].append(callback)


class CreditCard(object):
    expense = CallbackProperty(0)
    
def limit_breach_alert(value):
    if value > 10000000:
        print("You have maxed out your credit card.")


In [34]:
card = CreditCard()
CreditCard.expense.register_callback(card, limit_breach_alert)

card.expense = 20000000

You have maxed out your credit card.


In [35]:
card.expense = 100
print("Balance is %s" % card.expense)

Balance is 100


In [36]:
# 1.1.14	Lazy Properties of Descriptors
############################################

import random, time
class Universe:
    def answer_to_life(self):
        time.sleep(5) # Consider this as the thinking time. 
        return 42

myUniverse = Universe()
print(myUniverse.answer_to_life())
print(myUniverse.answer_to_life())
print(myUniverse.answer_to_life())


42
42
42


In [ ]:
# Using cached properties descriptors
######################################

import random, time

class CachedProperty:
    def __init__(self, func):
        self.func = func
        self.fname = func.__name__

    def __get__(self, instance, type=None) -> object:
        instance.__dict__[self.fname] = self.func(instance)
        return instance.__dict__[self.fname]

class Universe:
    @CachedProperty
    def answer_to_life(self):
        time.sleep(5)
        return 42

myUniverse = Universe()
print(myUniverse.answer_to_life())
print(myUniverse.answer_to_life())
print(myUniverse.answer_to_life())

In [40]:
# 1.1.15	Code in accordance to the D.R.Y. principle
#########################################################

from math import factorial
def is_prime(x):
    return factorial(x - 1)  % x == x - 1


class PrimeDeals:
    def __init__(self):
        self._prop_one = 0
        self._prop_two = 0
        self._prop_three = 0
        self._prop_four = 0
        self._prop_fine = 0

    @property
    def prop_one(self):
        return self._prop_one

    @prop_one.setter
    def prop_one(self, number):
        self._prop_one = number if is_prime(number) else 0

    @property
    def prop_two(self):
        return self._prop_two

    @prop_two.setter
    def prop_two(self, number):
        self._prop_two = number if is_prime(number) else 0

    @property
    def prop_three(self):
        return self._prop_three

    @prop_three.setter
    def prop_three(self, number):
        self._prop_three = number if is_prime(number) else 0

    @property
    def prop_four(self):
        return self._prop_four

    @prop_four.setter
    def prop_four(self, number):
        self._prop_four = number if is_prime(number) else 0


deals = PrimeDeals()
deals.prop_one = 7
deals.prop_two = 4

print(deals.prop_one)
print(deals.prop_two)


7
0


In [41]:
# Using Prime Number Descriptors

class PrimeNumber:
    def __set_name__(self, owner, fname):
        self.fname = fname

    def __get__(self, instance, type=None) -> object:
        return instance.__dict__.get(self.fname) or 0

    def __set__(self, instance, val) -> None:
        instance.__dict__[self.fname] = (val if is_prime(val) else 0)


class PrimeDeals:
    prop_one = PrimeNumber()
    prop_two = PrimeNumber()
    prop_three = PrimeNumber()
    prop_four = PrimeNumber()

In [42]:
deals = PrimeDeals()
deals.prop_one = 7
deals.prop_two = 4

print(deals.prop_one)
print(deals.prop_two)

7
0
